In [1]:
import ultralytics
ultralytics.__version__

'8.1.20'

In [2]:
import cv2
import pandas as pd
from ultralytics import YOLO
from tracking_program import*
import time

# We use load and use the yolo8s model:
model = YOLO('yolov8s.pt')

In [3]:
# Array for storing different classes of vehicles:
vehicle_class_list = ['person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus', 'train', 'truck', 'boat', 'traffic light', 'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow', 'elephant', 'bear', 'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie', 'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball', 'kite', 'baseball bat', 'baseball glove', 'skateboard', 'surfboard', 'tennis racket', 'bottle', 'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple', 'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza', 'donut', 'cake', 'chair', 'couch', 'potted plant', 'bed', 'dining table', 'toilet', 'tv', 'laptop', 'mouse', 'remote', 'keyboard', 'cell phone', 'microwave', 'oven', 
              'toaster', 'sink', 'refrigerator', 'book', 'clock', 'vase', 'scissors', 'teddy bear', 'hair drier', 'toothbrush']

In [4]:
tracker = Tracker()

# Initiate the frame count to 0:
frame_count = 0

In [5]:
# Video list for testing:
video = ['highway_mini.mp4', 'highway.mp4']
video_choice = 1

In [6]:
# Load the video for testing:
cap = cv2.VideoCapture(video[video_choice])

<img src = "a.png">

In [7]:
# Down dictionary created for (###)
# This dictionary will store all bounding box id and the data regarding the bounding box which touch the red line
# down = {}
# up = {}

up = {}
down = {}
# touch_count = {}

down = {}
up = {}

# Reading all the frames of the video, and resizing the framess size:
while True:
    ret, frame = cap.read()

    # End of the video, break out of the while loop:
    if not ret:
        break

    # Else, count the frame:
    frame_count += 1

    # Resize the frame:
    frame = cv2.resize(frame, (1020, 500))

    # Inferencing the model, feed the frame to it for prediction:
    results = model.predict(frame)

    # Showing the detetion reult in form of box with text label:
    a = results[0].boxes.data

    # a is running in GPU, need to deteach it, put it on cpu for drawing the line:
    # a contain the prediction resultas below graph show
    # It 0 - 3 is for the bounding box, 4 is the highest probability, 5 is the highest probability category:
    a = a.detach().cpu().numpy()

    # It is can show what is inside a:
    px = pd.DataFrame(a).astype('float')
    # print(px)

    # Another seperate list for storing only bounding box data related to truck and car class:
    list = []

    # We only want to store the prediction (stored in ox dataframe already!!!) related to truck and car classes:
    for index, row in px.iterrows():

        # Four values recording the bounding box data:
        x1 = int(row[0])
        y1 = int(row[1])
        x2 = int(row[2])
        y2 = int(row[3])

        # Highest probability classes:
        d = int(row[5])
        # Convert the numeric class index to corresponding string:
        c = vehicle_class_list[d]

        # Only store the data related to truck or car
        if 'car' or 'truck' in c:
            # Attach the result to the list:
            list.append([x1,y1,x2,y2])
            # print(list)



    # *****IMP****** 
    # Codes for tracking the object:
    # Storing the bounding box id using the bounding box record for car and truck class in the list:
    bounding_box_id = tracker.update(list)
    # print(bounding_box_id)

    # Showing the bounding box
    for bbox in bounding_box_id:
        # x, y, w, h
        x3, y3, x4, y4, id = bbox

        # Calculate the horizontal and vertical center again:
        cx = int(x3+x4)//2
        cy = int(y3+y4)//2

        # Draw the center of bounding box:
        # cv2.circle(frame,(cx,cy), 4, (0, 0, 255), -1)           (***)
        # Draw the actual bounding box:
        # cv2.rectangle(frame,(x3,y3), (x4, y4), (0, 0, 255), 2)  (***)

        # New condition (***):
        red_line_y = 198
        blue_line_y = 268
        offset = 7
        
        
        # Condition for touching the red line:
        # if red_line_y < (cy + offset) and red_line_y > (cy - offset):

            # New up dictionary!!!
        #     down[id] = time.time()

    
            # We only draw bounding box for the car touch the red line before (the car's bounding box id in down list already):
            # After the car pass through the middle of the red line, the center will not be shown
            # But still, its center was recorded in the down dictionary!!!
         #    if id in down:
                # Draw center for the car, and put its id down:
         #        cv2.circle(frame,(cx,cy), 4, (0, 0, 255), -1)    
         #        cv2.putText(frame, str(id), (cx, cy), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 255), 2) 


        # Similarl, we want to setup the condition for the blue line:
        # Condition for touching the red line:
        # if blue_line_y < (cy + offset) and blue_line_y > (cy - offset):

            # Put the center to the down list, that means we record this car passing the red line!!!
            # down[id] = cy
            # However, to find out the speed, we don't need the cy, we need to record time when touching the line instead:
        #     up[id] = time.time()

    
            # We only draw bounding box for the car touch the red line before (the car's bounding box id in down list already):
            # After the car pass through the middle of the red line, the center will not be shown
            # But still, its center was recorded in the down dictionary!!!
        #     if id in up:
                # Draw center for the car, and put its id down:
        #         cv2.circle(frame,(cx,cy), 4, (0, 0, 255), -1)    
        #         cv2.putText(frame, str(id), (cx, cy), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 255), 2) 


        ### Finally, we merge the code for both blue and red part
        if red_line_y < (cy + offset) and red_line_y > (cy - offset):
            down[id] = time.time()

        if id in down:
            if blue_line_y < (cy + offset) and blue_line_y > (cy - offset):
                elapsed_time = time.time() - down[id]
                distance = 10 # meters  (Distance between the 2 lines is 10 meters )
                a_speed_ms = distance / elapsed_time
                a_speed_kh = a_speed_ms * 3.6
                cv2.circle(frame,(cx,cy),4,(0,0,255),-1)
                cv2.rectangle(frame, (x3, y3), (x4, y4), (0, 255, 0), 2)  # Draw bounding box
                cv2.putText(frame,str(id),(x3,y3),cv2.FONT_HERSHEY_COMPLEX,0.6,(255,255,255),1)
                cv2.putText(frame,str(int(a_speed_kh))+'Km/h',(x4,y4),cv2.FONT_HERSHEY_COMPLEX,0.8,(0,255,255),2)

        # or (blue_line_y < (cy + offset) and blue_line_y > (cy - offset)

        elif blue_line_y < (cy + offset) and blue_line_y > (cy - offset):
            up[id] = time.time()

        if id in up:
            if red_line_y < (cy + offset) and red_line_y > (cy - offset):
                elapsed_time = time.time() - up[id]
                distance = 10 # meters  (Distance between the 2 lines is 10 meters )
                a_speed_ms = distance / elapsed_time
                a_speed_kh = a_speed_ms * 3.6
                cv2.circle(frame,(cx,cy),4,(0,0,255),-1)
                cv2.rectangle(frame, (x3, y3), (x4, y4), (0, 255, 0), 2)  # Draw bounding box
                cv2.putText(frame,str(id),(x3,y3),cv2.FONT_HERSHEY_COMPLEX,0.6,(255,255,255),1)
                cv2.putText(frame,str(int(a_speed_kh))+'Km/h',(x4,y4),cv2.FONT_HERSHEY_COMPLEX,0.8,(0,255,255),2)

        



        # if red_line_y<(cy+offset) and red_line_y > (cy-offset):
        #    down[id]=time.time()   # current time when vehichle touch the first line
        # if id in down:
          
        #    if blue_line_y<(cy+offset) and blue_line_y > (cy-offset):
        #      elapsed_time=time.time() - down[id]  # current time when vehicle touch the second line. Also we a re minusing the previous time ( current time of line 1)

        #      distance = 10 # meters - distance between the 2 lines is 10 meters
        #      a_speed_ms = distance / elapsed_time
        #      a_speed_kh = a_speed_ms * 3.6  # this will give kilometers per hour for each vehicle. This is the condition for going downside
        #      cv2.circle(frame,(cx,cy),4,(0,0,255),-1)
        #      cv2.rectangle(frame, (x3, y3), (x4, y4), (0, 255, 0), 2)  # Draw bounding box
        #      cv2.putText(frame,str(id),(x3,y3),cv2.FONT_HERSHEY_COMPLEX,0.6,(255,255,255),1)
        #      cv2.putText(frame,str(int(a_speed_kh))+'Km/h',(x4,y4 ),cv2.FONT_HERSHEY_COMPLEX,0.8,(0,255,255),2)

                
        #####going UP#####     
        # if blue_line_y<(cy+offset) and blue_line_y > (cy-offset):
        #    up[id]=time.time()
        # if id in up:

        #  if red_line_y<(cy+offset) and red_line_y > (cy-offset):
        #      elapsed1_time=time.time() - up[id]
             # formula of speed= distance/time  (distance travelled and elapsed time) Elapsed time is It represents the duration between the starting point and the ending point of the movement.

        #      distance1 = 10 # meters  (Distance between the 2 lines is 10 meters )
        #      a_speed_ms1 = distance1 / elapsed1_time
        #      a_speed_kh1 = a_speed_ms1 * 3.6
        #      cv2.circle(frame,(cx,cy),4,(0,0,255),-1)
        #      cv2.rectangle(frame, (x3, y3), (x4, y4), (0, 255, 0), 2)  # Draw bounding box
        #      cv2.putText(frame,str(id),(x3,y3),cv2.FONT_HERSHEY_COMPLEX,0.6,(255,255,255),1)
        #      cv2.putText(frame,str(int(a_speed_kh1))+'Km/h',(x4,y4),cv2.FONT_HERSHEY_COMPLEX,0.8,(0,255,255),2)
                


    # Try to see what is inside down dictionary
    # print(down)


    # Next, to find out the speed of the object, two line need to be drawn on the ground (the lines appear on each frame):
    # Color code:
    text_color = (255,255,255)  # white color for text
    red_color = (0, 0, 255)  # (B, G, R)   
    blue_color = (255, 0, 0)  # (B, G, R)
    green_color = (0, 255, 0)  # (B, G, R)  

    # First line:
    # Starting coordinate and end line coordinate (y-coord unchange!!! Since it is a horizontal line)
    cv2.line(frame, (172, 198), (774, 198), red_color, 3)
    cv2.putText(frame, ('Red line'), (172, 198), cv2.FONT_HERSHEY_SIMPLEX, 0.5, text_color, 1, cv2.LINE_AA)
    
    # Second line:
    cv2.line(frame, (8, 268), (927, 268), blue_color, 3)
    cv2.putText(frame, ('Blue line'), (8, 268), cv2.FONT_HERSHEY_SIMPLEX, 0.5, text_color, 1, cv2.LINE_AA)



    # Now, all vehicle will be deteced and drawn bounding box for them, but we don't want this:(
    # We just want to detect the vehicles that pass through the lines (either red or blue!!!)
    # We only want to put the center dot and bounding box on the vehicle pass through the lines only!!!
    # Thus, we are commenting the (***) lines!!!
    # (VERY IMP!!!)
    # New condition (###)

    
    
    

    # If reaching the end of the video break out pf the while loop
    cv2.imshow("frames", frame)

    if cv2.waitKey(16)&0xFF == 27:
          break
    # if cv2.waitKey(0.1):
    #        break


cap.release()
cv2.destroyAllWindows()


0: 320x640 8 cars, 2 trucks, 116.0ms
Speed: 3.0ms preprocess, 116.0ms inference, 141.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 8 cars, 2 trucks, 9.0ms
Speed: 2.0ms preprocess, 9.0ms inference, 2.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 7 cars, 2 trucks, 8.0ms
Speed: 2.0ms preprocess, 8.0ms inference, 3.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 6 cars, 2 trucks, 8.0ms
Speed: 2.0ms preprocess, 8.0ms inference, 2.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 8 cars, 2 trucks, 8.0ms
Speed: 2.0ms preprocess, 8.0ms inference, 2.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 8 cars, 2 trucks, 9.0ms
Speed: 1.0ms preprocess, 9.0ms inference, 2.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 8 cars, 2 trucks, 9.0ms
Speed: 1.0ms preprocess, 9.0ms inference, 2.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 8 cars, 2 trucks, 9.0ms
Speed: 1.0ms preprocess, 9.0ms inf

<b>cx and cy is the center of the bounding box:

<img src = "cxcy.png">

In [8]:
3 // 2

1

<b>Two lines

<img src = "twolines.png">

<h1><b>The read_line_y is the y coord of red line</h1>

<img src = "rly.png">

<h1><b>Speed calculation formula</h1>

<img src = "speedfor.png">

<b>Assuming the distance between red and blue line is 10m